In [1]:
#Imports
from io import BytesIO
import os
import open3d as o3d
import random
import requests
import tarfile
import numpy as np
from scipy.spatial import distance
from scipy.spatial.distance import cdist, pdist, squareform

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def gromov_wasserstein(pc1: np.ndarray, pc2: np.ndarray) -> float:
    def dist_ecc_fast(ecc, u):
        return (np.mean(ecc <= u))

    out = 0
    # Konvertiere die Punktwolken in NumPy-Arrays
    pc1 = np.asarray(pc1.points)
    pc2 = np.asarray(pc2.points)

    # Reshape input matrices if necessary
    if pc1.ndim == 1:
        pc1 = pc1.reshape(-1, 1)
    if pc2.ndim == 1:
        pc2 = pc2.reshape(-1, 1)

    ecc1 = squareform(pdist(pc1)).mean(0)
    ecc2 = squareform(pdist(pc2)).mean(0)
    unique_ecc = np.unique(np.concatenate((ecc1, ecc2)))
    for i in range(unique_ecc.shape[0] - 1):
        u = unique_ecc[i]
        out += (unique_ecc[i + 1] - unique_ecc[i]) * np.abs(dist_ecc_fast(ecc1, u) - dist_ecc_fast(ecc2, u))

    return (0.5 * out)

In [3]:
def chamfer_distance(pc1: np.ndarray, pc2: np.ndarray) -> float:
    dist = cdist(pc1, pc2)
    ch_dist = (np.min(dist, axis=1).mean() + np.min(dist, axis=0).mean()) / 2
    return ch_dist

In [4]:
def average_ratio(pc1: np.ndarray, pc2: np.ndarray, Dist_list: list) -> float:
    d = cdist(pc1, pc2)
    d0 = d.min(0)
    d1 = d.min(1)

    avr = 0
    for i, dist in enumerate(Dist_list):
        avr += (i + 1) * ((d1 <= dist).sum() / pc1.shape[0] + (d0 <= dist).sum() / pc2.shape[0])
    return avr / (len(Dist_list) ** 2 + len(Dist_list))

In [5]:
def ratio(pc1: np.ndarray, pc2: np.ndarray, thr_list=np.array([0.1, 0.5, 1, 2, 4])) -> list:
    d = np.min(cdist(pc1, pc2), axis=1)
    r = (d <= thr_list[:, None]).mean(1)
    return thr_list, r

In [6]:
def create_pcd_from_mesh(mesh):
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh])
    # distribute dots evenly on the surface
    return mesh.sample_points_uniformly(500)

In [7]:
def load_model(link, path):
    # http://ycb-benchmarks.s3-website-us-east-1.amazonaws.com/
    response = requests.get(link)
    tgz_data = BytesIO(response.content)
    # set the current working directory to the script's directory
    script_directory = os.path.dirname(os.path.abspath(__file__))
    os.chdir(script_directory)
    with tarfile.open(fileobj=tgz_data, mode="r:gz") as tar_ref:
        tar_ref.extractall(script_directory)
    # join paths
    model_path = os.path.join(script_directory, path, "clouds", "merged_cloud.ply")
    # load pointcloud
    pcd = o3d.io.read_point_cloud(model_path)
    return pcd

In [8]:
def load_cad_model(model):
    # load model generated in freecad
    return o3d.io.read_point_cloud(model)

In [9]:
def visualize_model(model):
    o3d.visualization.draw_geometries([model])

In [10]:
def get_num_points(model):
    print(len(model.points))

In [11]:
def create_pointcloud_from_coordinates(coordinates):
    # create point cloud with coordinates
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(coordinates)
    return pcd

In [12]:
def get_coordinates(model):
    coordinates = [list(point) for point in model.points]
    # print(coordinates[:50])
    return coordinates

In [13]:
def random_downsampling(model, endpoints):
    # get coordinates of the models
    coordinates = get_coordinates(model)
    # select random points for downsampling
    for i in range(len(coordinates) - endpoints):
        rannumb = random.randint(0, len(coordinates) - 1)
        del coordinates[rannumb]
    return coordinates

In [14]:
def farthest_point_sampling1(model, num_points_keep):
    coordinates = np.array(get_coordinates(model))
    retVal = []
    # to make runs comparable
    random.seed(13)
    # generate "random" int
    randint = random.randint(0, len(coordinates) - 1)
    # select random point from model
    retVal = np.append(retVal, coordinates[randint])
    # delete chosen point from original model after it was added to the downsampled cloud
    coordinates = np.delete(coordinates, randint, axis=0)
    while len(retVal) < num_points_keep:
        retValnp = np.array(retVal)
        # berechne die Entfernungen der ausgewählten Punkte zu den ausgewählten Punkten
        eucl_distances = distance.cdist(retValnp, coordinates, 'euclidean')
        # finde den weitesten Punkt heraus
        # füge den am weitesten entfernten Punkt der Liste hinzu
        retVal = np.append(retValnp, np.max(eucl_distances, axis=0))
        # punkt aus der Liste mit den ursprünglichen Koordinaten löschen
        min_distance_index = np.argmax(eucl_distances)
        coordinates = np.delete(coordinates, min_distance_index, axis=0)
    return retVal

In [15]:
def farthest_point_sampling(model, num_points_keep):
    coordinates = np.array(get_coordinates(model))
    retVal = []
    # to make runs comparable
    random.seed(13)
    # generate "random" int
    randint = random.randint(0, len(coordinates) - 1)
    # select random point from model
    retVal.append(coordinates[randint])
    # delete chosen point from original model after it was added to the downsampled cloud
    coordinates = np.delete(coordinates, randint, axis=0)
    while len(retVal) < num_points_keep:
        # Berechne die euklidischen Distanzen der ausgewählten Punkte zu den verbleibenden Punkten
        eucl_distances = distance.cdist(retVal, coordinates, 'euclidean')
        # Finden Sie den Punkt mit der größten Mindestdistanz
        min_mindist = np.min(eucl_distances, axis=0)
        # Finden Sie den Index des Punktes mit der größten Mindestdistanz
        max_min_distance_index = np.argmax(min_mindist)
        # Fügen Sie den am weitesten entfernten Punkt der Liste hinzu
        retVal.append(coordinates[max_min_distance_index])
        # Entfernen Sie den ausgewählten Punkt aus den verbleibenden Koordinaten
        coordinates = np.delete(coordinates, max_min_distance_index, axis=0)
    return np.array(retVal)

In [16]:
# built in function von open3d?
def radius_outlier_removal_call(model):
    return model.remove_radiues_outlier(nb_points=5, radius=0.05)

In [41]:
# with median -> aggregation method as parameter??
def create_voxel_grid(model, voxel_size):
    model_points = np.array(get_coordinates(model))
    min_bound = np.min(model_points, axis=0)
    max_bound = np.max(model_points, axis=0)

    dimensions = np.ceil((max_bound - min_bound) / voxel_size).astype(int)

    voxelgrid = np.zeros(dimensions)

    for point in model_points:
        voxel_coordinates = ((point - min_bound) / voxel_size).astype(int)
        # -1 needed in order to avoid index out of bounds
        voxelgrid[tuple(voxel_coordinates - 1)] += 1
    # visualize voxel grid to check if its correct
    # convert voxelgrid to open3d Voxelgrid
    o3d_voxelgrid = o3d.geometry.VoxelGrid.create_from_point_cloud(input=model, voxel_size=voxel_size)
    #o3d.visualization.draw_geometries([o3d_voxelgrid])
    return o3d_voxelgrid


def voxel_filter(model, voxelgrid, voxel_size):
    # list where downsampled points will be saved
    downsampled_points = []
    # iterate over all voxel in the voxelgrid
    for voxel in voxelgrid.get_voxels():
        # get bounds of the voxel
        downsampled_points.extend(is_point_in_voxel(model, voxelgrid, voxel, voxel_size))
    downsampled_points = np.asarray(downsampled_points)
    return o3d.utility.Vector3dVector(downsampled_points)


def aggregate_points(points):
    # Aggregate the points by averaging, taking into account the z coordinate
    if len(points) == 0:
        return points
    aggregated_points = []
    aggregated_points.append(np.mean(points, axis=0))
    return aggregated_points


def is_point_in_voxel(model, voxelgrid, voxel, voxel_size):
    # get center point and see whether a point lies within the given distance/2 of the voxel size from the center
    voxel_center = voxelgrid.get_voxel_center_coordinate(voxel.grid_index)
    points_in_voxel = []
    half_size = voxel_size / 2.0
    # Überprüfe, welche Punkte innerhalb des Voxels liegen
    for point in model.points:
        if np.all(np.abs(point - voxel_center) <= half_size):
            points_in_voxel.append(point)
    points_in_voxel = aggregate_points(points_in_voxel)
    # print(points_in_voxel)
    return points_in_voxel

def create_points_from_voxel(voxel_model):
    # Vektoren in Numpy Array konvertieren
    vector_array = np.asarray(voxel_model)
    
    # Open3D Punktewolke erstellen
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(vector_array)
    
    return point_cloud


In [18]:
import inspect

def point_cloud_to_ply(point_cloud, file_name): 
    # safe downsampled point cloud as ply data
    o3d.io.write_point_cloud(file_name, point_cloud)

In [19]:
import logging
# Logging
logging.basicConfig(filename='downsampling.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# Pfad zum Speichern der Bilder
output_dir = 'point_cloud_images'
os.makedirs(output_dir, exist_ok=True)
#except Exception as e: logging.error(f'Iteration {i+1}: Fehler aufgetreten - {e}')

In [20]:
# to do lege die Bilder mit relaitven Pfad im Repo an!
cone = load_cad_model(r"cone.ply")
sphere = load_cad_model(r"sphere.ply")
cube = load_cad_model(r"cube.ply")
complex_cube = load_cad_model(r"complexCube.ply")
hollow_cone = load_cad_model(r"hollowCone.ply")
complex_sphere = load_cad_model(r"complexSphere.ply")
pencil = load_cad_model(r"pencil_fein.ply")
# source: https://sketchfab.com/3d-models/davis-teapot-materialcleanup-547971eaf21d43f2b6cfcb6be0e7bf11
teapot = load_cad_model(r"teapot.ply")
# source: https://sketchfab.com/3d-models/book-ba04f5ac66194341bc7d437fb6c94674
book = load_cad_model(r"book.ply")

In [21]:
# Laufzeittests

In [22]:
from open3d.examples.open3d_example import draw_registration_result
# Auswertung ICP
# Transformiere die Ziel-Punktwolke zur besseren Demonstration
# ich kann hier auch meine Matplotlib Implementierung verwenden!
def icp_algorithm(source, target):
    # Transformierung der Target Punktewolke
    transformation = np.array([[0.86, 0.5, 0.1, 0.5],
                               [-0.5, 0.86, 0.1, 0.5],
                               [0.0, -0.1, 0.99, 0.5],
                               [0.0, 0.0, 0.0, 1.0]])
    target = target.transform(transformation)
    
    # ICP-Algorithmus ausführen
    threshold = 0.02  # Maximale Entfernung für die Zuordnung von Punkten
    initial_transformation = np.identity(4)  # Initiale Schätzung der Transformation
    
    # Open3D ICP Algorithmus
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, initial_transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    
    # Ergebnis grafisch ausgeben lassen
    draw_registration_result(source, target, reg_p2p.transformation)
    
    # evaluierung wie gut der ICP lief
    evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, transformation)
    print(evaluation)


In [23]:
# Auswertung der quantitativen Methoden

In [24]:
import time

#Test für Stabilität der Ergebnisse
# Anzahl der Wiederholungen
num_iterations = 20
output_dir = 'point_cloud_images'
os.makedirs(output_dir, exist_ok=True)

model_array = [cone, cube,sphere, hollow_cone, complex_sphere, pencil, teapot, book]
for i in range(model_array):
    for i in num_iterations:
        try:
           # Zeitmessung starten
           start_time = time.time()
           # Methodenaufrufe
           rd = random_downsampling(i, 200)
           # Zeitmessung beenden
           end_time = time.time()
           #  verstrichene Zeit loggen
           elapsed_time = end_time - start_time
           # bei len muss dann noch die aktuell getestete Methode rein -> maybe noch die Ausgangsgröße der
           # Punktewolke mitloggen?
           logging.info(f'Iteration {i+1}: Downsampling durchgeführt, verbleibende Punkte: {len(rd)}, Rechenzeit: {elapsed_time:.4f} Sekunden')
           # Bildspeicherung! -> nachschauen was durch das Reshapen hier passiert!
           point_cloud_to_ply(rd)
           #logging.info(f'Iteration {i+1}: Bild gespeichert als {image_path}')
           
        except Exception as e:
            logging.error(f'Iteration {i+1}: Fehler aufgetreten - {e}')

logging.info('Alle Iterationen abgeschlossen.')

TypeError: 'list' object cannot be interpreted as an integer

Sphere Test

In [ ]:
visualize_model(sphere)
sphere_rd= random_downsampling(sphere, int(len(sphere.points)/10 * 4))
sphere_rd_pc = o3d.geometry.PointCloud()
sphere_rd_pc.points = o3d.utility.Vector3dVector(sphere_rd)
#sphere_rd = point_cloud_to_ply(sphere_rd_pc, "sphere_rd.ply")
icp_algorithm(sphere, sphere_rd_pc) # nehme ich hier bei beiden die gedownsamplete Punktewolke?

Complex Sphere Test

In [ ]:
visualize_model(complex_sphere)
complex_sphere_rd= random_downsampling(complex_sphere, int(len(complex_sphere.points)/10 * 2))
complex_sphere_rd_pc = o3d.geometry.PointCloud()
complex_sphere_rd_pc.points = o3d.utility.Vector3dVector(complex_sphere_rd)
#sphere_rd = point_cloud_to_ply(sphere_rd_pc, "sphere_rd.ply")
icp_algorithm(complex_sphere_rd_pc, complex_sphere_rd_pc) # nehme ich hier bei beiden die gedownsamplete Punktewolke?

Teapot Test

In [ ]:
visualize_model(teapot)
teapot_rd= random_downsampling(teapot, int(len(teapot.points)/10 * 2))
teapot_rd_pc = o3d.geometry.PointCloud()
teapot_rd_pc.points = o3d.utility.Vector3dVector(teapot_rd)
#sphere_rd = point_cloud_to_ply(sphere_rd_pc, "sphere_rd.ply")
icp_algorithm(teapot_rd_pc, teapot_rd_pc) # nehme ich hier bei beiden die gedownsamplete Punktewolke?

In [42]:
teapot_vg = create_voxel_grid(teapot, voxel_size=0.2)
teapot_voxel_filter = voxel_filter(teapot,teapot_vg, voxel_size=0.2)
# Voxelgrid in Pointcloud konvertieren -> 12.414 Punkte original; 1305 Punkte Endergebnis
visualize_model(create_points_from_voxel(teapot_voxel_filter))

PointCloud with 1305 points.